# Batch Scoring on IBM Cloud Pak for Data (ICP4D)

We are going to use this notebook to create and/or run a batch scoring job against a model that has previously been created and deployed to the Watson Machine Learning (WML) instance on Cloud Pak for Data (CP4D).

## 1.0 Install required packages


There are a couple of Python packages we will use in this notebook. First we make sure the Watson Machine Learning client v3 is removed (its not installed by default) and then install/upgrade the v4 version of the client (this package is installed by default on CP4D).
- WML Client: https://wml-api-pyclient-dev-v4.mybluemix.net/#repository

In [1]:
!pip uninstall watson-machine-learning-client -y
!pip install --user watson-machine-learning-client-v4==1.0.99 --upgrade | tail -n 1

In [2]:
import json
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## 2.0 Create Batch Deployment Job

### 2.1 Instantiate Watson Machine Learning Client

To interact with the local Watson Machine Learning instance, we will be using the Python SDK. 

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

<font color='red'>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

`
wml_credentials = {
                   "url": "https://zen.clusterid.us-south.containers.appdomain.cloud",
                   "username": "cp4duser",
                   "password" : "cp4dpass",
                   "instance_id": "wml_local",
                   "version" : "2.5.0"
                  }
`
#### NOTE: Make sure that there is no trailing forward slash `/` in the `url`

In [3]:
# Be sure to update these credentials before running the cell.
wml_credentials = {
                   "url": "https://zen********.ibmcloudpack.com",
                   "username": "*******",
                   "password" : "*******",
                   "instance_id": "wml_local",
                   "version" : "2.5.0"
                  }

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
wml_client.spaces.list()

------------------------------------  -----------------------  ------------------------
GUID                                  NAME                     CREATED
dc1ff2f4-5dd6-4957-aae1-23b31f5936f2  ChurnDepSpace-Demouser1  2020-06-03T22:10:22.847Z
------------------------------------  -----------------------  ------------------------


### 2.2 Find Deployment Space

We will try to find the `GUID` for the deployment space you want to use and set it as the default space for the client.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update with the value with the name of the deployment space where you have created the batch deployment (one of the values in the output from the cell above).

In [5]:
# Be sure to update the name of the space with the one you want to use.
DEPLOYMENT_SPACE_NAME = 'ChurnDepSpace-Demouser1'

In [6]:
all_spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["guid"]
        print("\nDeployment Space GUID: ", space_id)

if space_id is None:
    print("WARNING: Your space does not exist. Create a deployment space before proceeding.")
    # We could programmatically create the space.
    #space_id = wml_client.spaces.store(meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]


Deployment Space GUID:  dc1ff2f4-5dd6-4957-aae1-23b31f5936f2


In [7]:
# Now set the default space to the GUID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(space_id)

'SUCCESS'

In [8]:
# These are the models and deployments we currently have in our deployment space.
wml_client.repository.list_models()
wml_client.deployments.list()

------------------------------------  -------------------------------------------------  ------------------------  --------------
GUID                                  NAME                                               CREATED                   TYPE
d62ab2e7-9a9a-49ed-ad7b-7c46a6e0663e  ChurnAutoAI06042020v1 - P3 XGBClassifierEstimator  2020-06-05T01:25:53.002Z  wml-hybrid_0.1
ca2426ba-80c9-4acb-b9c0-4053e4faa83b  ChurnSpark0604202v1                                2020-06-05T00:57:16.002Z  mllib_2.3
------------------------------------  -------------------------------------------------  ------------------------  --------------
------------------------------------  ---------------------  -----  ------------------------  -------------
GUID                                  NAME                   STATE  CREATED                   ARTIFACT_TYPE
66566bf3-4e3c-47f6-8134-0885a69bf412  ChurnSparkBatchDepv1   ready  2020-06-05T01:28:08.941Z  model
4a11336f-fb43-41e5-be9e-be5b7a651cf4  ChurnSparkO

### 2.3 Find Batch Deployment

We will try to find the batch deployment which was created.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update with the name of the batch deployment.

In [9]:
DEPLOYMENT_NAME = 'ChurnSparkBatchDepv1'

In [10]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
deployment_details = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        deployment_details = deployment
        #print(json.dumps(deployment_details, indent=3))
        break

print("Deployment id: {}".format(deployment_uid))
wml_client.deployments.get_details(deployment_uid)

Deployment id: 66566bf3-4e3c-47f6-8134-0885a69bf412


{'metadata': {'parent': {'href': ''},
  'name': 'ChurnSparkBatchDepv1',
  'guid': '66566bf3-4e3c-47f6-8134-0885a69bf412',
  'description': '',
  'id': '66566bf3-4e3c-47f6-8134-0885a69bf412',
  'modified_at': '2020-06-05T01:28:08.941Z',
  'created_at': '2020-06-05T01:28:08.941Z',
  'href': '/v4/deployments/66566bf3-4e3c-47f6-8134-0885a69bf412',
  'space_id': 'dc1ff2f4-5dd6-4957-aae1-23b31f5936f2'},
 'entity': {'name': 'ChurnSparkBatchDepv1',
  'custom': {},
  'description': '',
  'batch': {'schedule': {}},
  'space': {'id': 'dc1ff2f4-5dd6-4957-aae1-23b31f5936f2',
   'href': '/v4/spaces/dc1ff2f4-5dd6-4957-aae1-23b31f5936f2'},
  'status': {'state': 'ready'},
  'asset': {'id': 'ca2426ba-80c9-4acb-b9c0-4053e4faa83b',
   'href': '/v4/models/ca2426ba-80c9-4acb-b9c0-4053e4faa83b?space_id=dc1ff2f4-5dd6-4957-aae1-23b31f5936f2'},
  'space_id': 'dc1ff2f4-5dd6-4957-aae1-23b31f5936f2',
  'hardware_spec': {'id': 'f3ebac7d-0a75-410c-8b48-a931428cc4c5',
   'name': 'XS',
   'num_nodes': 1}}}

### 2.4 Get Batch Test Data

We will load some data to run the batch predictions.

In [11]:
import pandas as pd

from project_lib import Project
project = Project.access()

batch_set = pd.read_csv(project.get_file('Telco-Customer-Churn-SmallBatchSet.csv'))
batch_set = batch_set.drop('customerID', axis=1)
batch_set.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,1,Yes,Yes,23,Yes,No,DSL,Yes,Yes,Yes,Yes,No,No,One year,No,Mailed check,76.15,1874.45
1,Male,0,No,No,2,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Bank transfer (automatic),25.20,20.20
2,Female,1,Yes,No,1,Yes,No,DSL,No,Yes,No,No,No,No,Month-to-month,No,Bank transfer (automatic),85.25,45.25
3,Female,1,Yes,Yes,66,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Bank transfer (automatic),89.90,7251.70
4,Male,0,No,No,5,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,59.70,316.90


### 2.5 Create Job

We can now use the information about the deployment and the test data to create a new job against our batch deployment. We submit the data as inline payload and want the results (i.e predictions) stored in a CSV file.

In [12]:
import time
timestr = time.strftime("%Y%m%d_%H%M%S")
job_payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': batch_set.columns.values.tolist(),
        'values': batch_set.values.tolist()
    }],
    wml_client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "type": "data_asset",
            "connection": {},
            "location": {
                "name": "batchres_{}_{}.csv".format(timestr,deployment_uid),
                "description": "results"
            }
    }
}

job = wml_client.deployments.create_job(deployment_id=deployment_uid, meta_props=job_payload)
job_uid = wml_client.deployments.get_job_uid(job)

print('Job uid = {}'.format(job_uid))

Job uid = 7cd9f54f-5ca1-474f-8231-60daba3e6007


In [13]:
wml_client.deployments.list_jobs()

------------------------------------  ------  ------------------------  ------------------------------------
JOB-UID                               STATE   CREATED                   DEPLOYMENT-ID
7cd9f54f-5ca1-474f-8231-60daba3e6007  queued  2020-06-05T01:30:47.248Z  66566bf3-4e3c-47f6-8134-0885a69bf412
------------------------------------  ------  ------------------------  ------------------------------------


## 3.0 Monitor Batch Job Status

The batch job is an async operation. We can use the identifier to track its progress. Below we will just poll until the job completes (or fails).

In [14]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)
            
job_details = poll_async_job(wml_client, job_uid)

{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'completed', 'running_at': '2020-06-05T01:31:44.000Z', 'completed_at': '2020-06-05T01:31:46.000Z'}


In [15]:
wml_client.deployments.list_jobs()

------------------------------------  ---------  ------------------------  ------------------------------------
JOB-UID                               STATE      CREATED                   DEPLOYMENT-ID
7cd9f54f-5ca1-474f-8231-60daba3e6007  completed  2020-06-05T01:30:47.248Z  66566bf3-4e3c-47f6-8134-0885a69bf412
------------------------------------  ---------  ------------------------  ------------------------------------


### 3.1 Check Results

With the job complete, we can see the predictions. 

In [16]:
wml_client.deployments.get_job_details()

{'resources': [{'metadata': {'parent': {'href': ''},
    'guid': '7cd9f54f-5ca1-474f-8231-60daba3e6007',
    'id': '7cd9f54f-5ca1-474f-8231-60daba3e6007',
    'modified_at': '2020-06-05T01:31:46.679Z',
    'created_at': '2020-06-05T01:30:47.248Z',
    'href': '/v4/deployment_jobs/7cd9f54f-5ca1-474f-8231-60daba3e6007',
    'space_id': 'dc1ff2f4-5dd6-4957-aae1-23b31f5936f2'},
   'entity': {'deployment': {'id': '66566bf3-4e3c-47f6-8134-0885a69bf412',
     'href': '/v4/deployments/66566bf3-4e3c-47f6-8134-0885a69bf412'},
    'space_id': 'dc1ff2f4-5dd6-4957-aae1-23b31f5936f2',
    'scoring': {'input_data': [{'fields': ['gender',
        'SeniorCitizen',
        'Partner',
        'Dependents',
        'tenure',
        'PhoneService',
        'MultipleLines',
        'InternetService',
        'OnlineSecurity',
        'OnlineBackup',
        'DeviceProtection',
        'TechSupport',
        'StreamingTV',
        'StreamingMovies',
        'Contract',
        'PaperlessBilling',
        'P

In [17]:
print(json.dumps(job_details, indent=2))

{
  "metadata": {
    "parent": {
      "href": ""
    },
    "guid": "7cd9f54f-5ca1-474f-8231-60daba3e6007",
    "id": "7cd9f54f-5ca1-474f-8231-60daba3e6007",
    "modified_at": "2020-06-05T01:31:46.679Z",
    "created_at": "2020-06-05T01:30:47.248Z",
    "href": "/v4/deployment_jobs/7cd9f54f-5ca1-474f-8231-60daba3e6007",
    "space_id": "dc1ff2f4-5dd6-4957-aae1-23b31f5936f2"
  },
  "entity": {
    "deployment": {
      "id": "66566bf3-4e3c-47f6-8134-0885a69bf412",
      "href": "/v4/deployments/66566bf3-4e3c-47f6-8134-0885a69bf412"
    },
    "space_id": "dc1ff2f4-5dd6-4957-aae1-23b31f5936f2",
    "scoring": {
      "input_data": [
        {
          "fields": [
            "gender",
            "SeniorCitizen",
            "Partner",
            "Dependents",
            "tenure",
            "PhoneService",
            "MultipleLines",
            "InternetService",
            "OnlineSecurity",
            "OnlineBackup",
            "DeviceProtection",
            "TechSupport",

## Congratulations, you have created and submitted a job for batch scoring !